# Directory, Libraries and Data

In [15]:
# Import libraries
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt

In [4]:
# Data
data = pd.read_csv('online_shoppers.csv', encoding='ISO-8859-1').dropna()
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


# Data Preparation

In [5]:
# Creating Sales column
data['sales'] = data['Quantity'] * data['UnitPrice']
data.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


In [16]:
# Dates - Transform Date variable
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data['InvoiceDate']

# Get last date available
snapshot_date = data['InvoiceDate'].max() + timedelta(days=1)
snapshot_date

Timestamp('2011-12-10 12:50:00')

In [17]:
# Aggregate on Customer Level
df = data.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x:(snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'sales': 'sum'
})

df.head()

,InvoiceDate,InvoiceNo,sales
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


In [18]:
# Create basket / Monetary variable
df['Monetary'] = df.sales / df.InvoiceNo
df.head(2)

,InvoiceDate,InvoiceNo,sales,Monetary
CustomerID,,,,
12346.0,326,2,0.0,0.000000
12347.0,2,182,4310.0,23.681319


In [19]:
# Dropping Sales Variable
df = df.drop(columns='sales')
df.head(0)

,InvoiceDate,InvoiceNo,Monetary
CustomerID,,,


In [20]:
# Changing variable names
df.rename(columns={'InvoiceDate': 'Recency',
                   "InvoiceNo": 'Frequency'}, inplace=True)
df.head(1)

,Recency,Frequency,Monetary
CustomerID,,,
12346.0,326,2,0.0


# RFM Preparation

In [22]:
# Create Frequency Groups
df['F'] = pd.qcut(x = df['Frequency'], q=4, labels=range(1, 5, 1))
df['M'] = pd.qcut(x = df['Monetary'], q =4, labels = range(1,5,1))
df['R'] = pd.qcut(x = df['Recency'], q = 4, labels= range(4, 0, -1))
df.head(3)

,Recency,Frequency,Monetary,F,M,R
CustomerID,,,,,,
12346.0,326,2,0.000000,1,1,1
12347.0,2,182,23.681319,4,4,4
12348.0,75,31,57.975484,2,4,2


In [23]:
# RFM Score
df['RFM'] = df[['R', 'F', 'M']].sum(axis=1)
df.head(2)

,Recency,Frequency,Monetary,F,M,R,RFM
CustomerID,,,,,,,
12346.0,326,2,0.000000,1,1,1,3
12347.0,2,182,23.681319,4,4,4,12


In [24]:
# Create the RFM function
def rfm_segment(df):
    if df['RFM'] >= 11:
        return 'SuperStar'
    elif ((df['RFM'] >= 8) and (df['RFM'] < 11)):
        return 'Future Champion'
    elif((df['RFM'] >= 6) and (df['RFM'] < 8)):
        return 'High Potential'
    else:
        return 'Low Relevance'
    

# RFM

In [25]:
# Apply RFM function
df['RFM_level'] = df.apply(rfm_segment, axis=1)
df.head()

,Recency,Frequency,Monetary,F,M,R,RFM,RFM_level
CustomerID,,,,,,,,
12346.0,326,2,0.000000,1,1,1,3,Low Relevance
12347.0,2,182,23.681319,4,4,4,12,SuperStar
12348.0,75,31,57.975484,2,4,2,8,Future Champion
12349.0,19,73,24.076027,3,4,3,10,Future Champion
12350.0,310,17,19.670588,1,3,1,5,Low Relevance


In [29]:
# Looking into segments
df.groupby('RFM_level').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'count']
})

Recency   Frequency   Monetary      
                       mean        mean       mean count
RFM_level                                               
Future Champion   32.007919  135.835973  30.169670  1768
High Potential   118.708869   39.213947  38.033024  1477
Low Relevance    221.483400   22.811421  -0.586430   753
SuperStar          9.973262  244.852941  45.483949   374

# Challenge Time Customer Data

## Libraries and Data

In [30]:
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt

In [33]:
data = pd.read_csv('customer_data.csv').dropna()
data.head()

,customer_id,revenue,most_recent_visit,number_of_orders,recency_days
0,22086,777,5/14/2006,9,232
1,2290,1555,9/8/2006,16,115
2,26377,336,11/19/2006,5,43
3,24650,1189,10/29/2006,12,64
4,12883,1229,12/9/2006,12,23


In [46]:
data['Monetary'] = data['revenue'] / data['number_of_orders']
data.head(2)


,customer_id,revenue,most_recent_visit,number_of_orders,recency_days,Monetary
0,22086,777,5/14/2006,9,232,86.333333
1,2290,1555,9/8/2006,16,115,97.187500


In [55]:
df = data.drop(columns=['revenue', 'most_recent_visit'])
df = df.set_index('customer_id')

In [56]:
df.rename(columns= {'number_of_orders': 'Frequency', 'recency_days': 'Recency'}, inplace=True)
df

,Frequency,Recency,Monetary
customer_id,,,
22086,9,232,86.333333
2290,16,115,97.187500
26377,5,43,67.200000
24650,12,64,99.083333
12883,12,23,102.416667
...,...,...,...
3249,10,31,99.800000
6686,8,187,96.375000
16418,9,154,112.888889


In [65]:
df['F'] = pd.qcut(x = df['Frequency'], q = 3, labels = range(1, 4, 1))
df['R'] = pd.qcut(x = df['Recency'], q = 3, labels = range(3, 0, -1))
df['M'] = pd.qcut(x = df['Monetary'], q=3, labels = range(1,4,1))
df.head()

,Frequency,Recency,Monetary,F,R,M,RFM,RFM_level
customer_id,,,,,,,,
22086,9,232,86.333333,2,1,1,5,Low Relevance
2290,16,115,97.187500,3,2,2,8,Future Champion
26377,5,43,67.200000,1,3,1,6,High Potential
24650,12,64,99.083333,3,3,2,9,Future Champion
12883,12,23,102.416667,3,3,2,9,Future Champion


In [66]:
df['RFM'] = df[['F', 'R', 'M']].sum(axis = 1)
df.head()

,Frequency,Recency,Monetary,F,R,M,RFM,RFM_level
customer_id,,,,,,,,
22086,9,232,86.333333,2,1,1,4,Low Relevance
2290,16,115,97.187500,3,2,2,7,Future Champion
26377,5,43,67.200000,1,3,1,5,High Potential
24650,12,64,99.083333,3,3,2,8,Future Champion
12883,12,23,102.416667,3,3,2,8,Future Champion


In [67]:
df['RFM_level'] = df.apply(rfm_segment, axis = 1)
df.head()

,Frequency,Recency,Monetary,F,R,M,RFM,RFM_level
customer_id,,,,,,,,
22086,9,232,86.333333,2,1,1,4,Low Relevance
2290,16,115,97.187500,3,2,2,7,High Potential
26377,5,43,67.200000,1,3,1,5,Low Relevance
24650,12,64,99.083333,3,3,2,8,Future Champion
12883,12,23,102.416667,3,3,2,8,Future Champion


In [69]:
df.groupby('RFM_level').agg({
    'Frequency': 'mean',
    'Recency': 'mean',
    'Monetary': ['mean', 'count']
}).round(1)


Frequency Recency Monetary       
                     mean    mean     mean  count
RFM_level                                        
Future Champion      12.8    80.1    108.3   6375
High Potential       10.6   141.8     98.5  17916
Low Relevance         7.8   267.7     86.8  15708